from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
#%tensorflow_version 2.x
#import tensorflow as tf
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

#https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=sXnDmXR7RDr2

# **MyAZNet**  


### **Libraries.** 

In [3]:
import os
import pathlib
import torch
import tarfile
import torchvision
import glob
import torch.nn as nn
from torch.optim import Adam
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

In [4]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [5]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(device)

cuda


In [7]:
#GPU



### **Parameters.** 

In [8]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
#Dataloader

#Path for training and testing directory
train_path='../MIT_split/train'
test_path='../MIT_split/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
    )

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=64, shuffle=True
    )

In [10]:
#Image categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [11]:
print(classes)

['Opencountry', 'coast', 'forest', 'highway', 'inside_city', 'mountain', 'street', 'tallbuilding']


In [12]:
#CNN Network
class MLP(torch.nn.Module):
    def __init__(self,num_classes=8):
        super(MLP,self).__init__()

        self.num_classes = num_classes

        #Conv1
        self.conv1=torch.nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3, padding=1)
        #Shape= (128,3,224,224)
        self.relu1=nn.ReLU()

        #Batch Norm
        self.bn1=torch.nn.BatchNorm2d(num_features=64)
        #Shape= (128,12,224,224)
        self.relu2=nn.ReLU()
        #Shape= (128,12,224,224)

        #MaxPool
        self.pool1=torch.nn.MaxPool2d(2,2)
        #Shape= (128,12,112,112)
        
        #Conv2
        self.conv2=torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3, padding=1)
        #Shape= (32,112,112)
        self.relu3=nn.ReLU()
        #Shape= (32,112,112)

        #Global Avg Pooling
        self.pool2=torch.nn.AvgPool2d(kernel_size=2)

        #Dense
        self.fc=torch.nn.Linear(in_features=64*56*56, out_features=num_classes)
        self.relu4=torch.nn.ReLU()

        #Dropout
        self.drop1 = torch.nn.Dropout(p=0.4)

        #Dense
        self.fc2=torch.nn.Linear(in_features=8,out_features=num_classes)
        self.softmax = torch.nn.Softmax(dim=1)

       
    #Feed forwad function
        
    def forward(self,input):

        output=self.conv1(input)
        output=self.relu1(output)

        output=self.bn1(output)
        output=self.relu2(output)

        output=self.pool1(output)

        output=self.conv2(output)
        output=self.relu3(output)

        output=self.pool2(output)

        output=output.view(output.size(0),-1)

        output=self.fc(output)
        output=self.relu4(output)

        output=self.drop1(output)

        output=output.view(output.size(0),-1)

        output=self.fc2(output)            

        output=self.softmax(output)
            
        return output

In [13]:
model=MLP(num_classes=8).to(device)

In [14]:
from torchsummary import summary
summary (model, input_size=(3,224,224))

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [15]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=200

In [17]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(train_count,test_count)

1881 807


In [19]:

#Model training and saving best model
#source: https://github.com/gaurav67890/Pytorch_Tutorials/blob/master/cnn-scratch-training.ipynb

#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, data in enumerate(train_loader,0):
        if torch.cuda.is_available():
            images, labels = data
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        # train_loss+= loss.cpu().data*images.size(0)
        train_loss += loss.item()*len(images)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))
  
    #Evaluation on testing dataset
    #model.eval()
    
    #test_accuracy=0.0
    #for i, (images,labels) in enumerate(test_loader):
    #    if torch.cuda.is_available():
    #        images=Variable(images.cuda())
    #        labels=Variable(labels.cuda())
            
    #    outputs=model(images)
    #    _,prediction=torch.max(outputs.data,1)
    #    test_accuracy+=int(torch.sum(prediction==labels.data))
    
    #test_accuracy=test_accuracy/test_count
    
    
    #print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    #if test_accuracy>best_accuracy:
    #    torch.save(model.state_dict(),'best_checkpoint.model')
    #    best_accuracy=test_accuracy

Epoch: 0 Train Loss: 2.080331530614079 Train Accuracy: 0.1876661350345561
Epoch: 1 Train Loss: 2.045182880095381 Train Accuracy: 0.227538543328017
Epoch: 2 Train Loss: 2.021876669006104 Train Accuracy: 0.25199362041467305
Epoch: 3 Train Loss: 2.0257877555697097 Train Accuracy: 0.2482721956406167
Epoch: 4 Train Loss: 1.991034397224617 Train Accuracy: 0.2812333864965444
Epoch: 5 Train Loss: 2.0062296678258154 Train Accuracy: 0.26741095162147793
Epoch: 6 Train Loss: 1.9851077025016528 Train Accuracy: 0.2881446039340776
Epoch: 7 Train Loss: 2.0253008406951922 Train Accuracy: 0.2477405635300372
Epoch: 8 Train Loss: 1.9782526141972316 Train Accuracy: 0.2961190855927698
Epoch: 9 Train Loss: 1.9929544885129387 Train Accuracy: 0.28017012227538546
Epoch: 10 Train Loss: 2.0173345982299593 Train Accuracy: 0.2557150451887294
Epoch: 11 Train Loss: 2.0424230105061913 Train Accuracy: 0.2307283359914939
Epoch: 12 Train Loss: 2.017232757372656 Train Accuracy: 0.2567783094098884
Epoch: 13 Train Loss: 2.0

KeyboardInterrupt: 

In [39]:
#

# **Hyperparametrization**

In [40]:
# list all data in history
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../logs/plots_full_data/accuracy_hyper.jpg')
#plt.close()

NameError: name 'history' is not defined

In [ ]:
  # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../logs/plots_full_data/loss_hyper.jpg')